<a href="https://colab.research.google.com/github/jiieunshin/lecture_deeplearning/blob/master/bert_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# 이는 구글 코랩으로 돌린 버전입니다. 그리고 기본 코드는 SKT Brain의 KoBERT를 그대로 가져왔고, 학습 및 테스트 데이터셋만 따로 준비한 것입니다.
# SKT Brain github 주소는 다음과 같습니다. https://github.com/SKTBrain/KoBERT
!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install transformers==3 # 최신 버전으로 설치하면 "Input: must be Tensor, not str" 라는 에러 발생
!pip install torch
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

     |████████████████████████████████| 46.9MB 64kB/s 
  Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1
     |████████████████████████████████| 348kB 5.1MB/s 
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp37-cp37m-linux_x86_64.whl size=595700 sha256=e6a860e802659f1a0f16ff6af932a048c499471c9ba378ba5d823abdb398f977
  Stored in directory: /root/.cache/pip/wheels/37/65/52/63032864a0f31a08b9a88569f803b5bafac8abd207fd7f7534
Successfully built gluonnlp
     |████████████████████████████████| 1.2MB 5.0MB/s 
     |████████████████████████████████| 757kB 4.3MB/s 
     |████████████████████████████████| 901kB 19.3MB/s 
     |████████████████████████████████| 3.0MB 25.8MB/s 
  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-xexu29px
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-xexu29px
  Created wheel for kobert:

In [2]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [3]:
##GPU 사용 시
device = torch.device("cuda:0")
bertmodel, vocab = get_pytorch_kobert_model()

[██████████████████████████████████████████████████]
[██████████████████████████████████████████████████]


In [4]:
# !pip install git+https://github.com/haven-jeon/PyKoSpacing.git
 
# from pykospacing import Spacing
# import re 

# save_text = []
# for idx, contents in enumerate(train_data['내용']):
#     pre_text = re.sub("[^가-힣ㄱ-ㅎㅏ-ㅣ\\s]", "", contents) 
#     spacing = Spacing()
#     save_text.append(spacing(pre_text)) 
 
# train_data['내용2'] = save_text
# train_data  
 
# word_cnt = train_data['내용2'].astype(str).apply(lambda x:len(x.split(' ')))
# train_data['word_cnt'] = word_cnt
 
# drop_ind=[]
# for ind, cnt in enumerate(train_data['word_cnt']):
#     if cnt <= 10:
#         drop_ind.append(ind)
 
# train_data.drop(train_data.index[drop_ind], inplace=True)
# train_data  


In [5]:
# save_text = []
# for idx, contents in enumerate(test_data['내용']):
#     pre_text = re.sub("[^가-힣ㄱ-ㅎㅏ-ㅣ\\s]", "", contents) 
#     spacing = Spacing()
#     save_text.append(spacing(pre_text)) 
 
# test_data['내용2'] = save_text
# test_data  
 
# word_cnt = test_data['내용2'].astype(str).apply(lambda x:len(x.split(' ')))
# test_data['word_cnt'] = word_cnt
 
# drop_ind=[]
# for ind, cnt in enumerate(test_data['word_cnt']):
#     if cnt <= 10:
#         drop_ind.append(ind)
 
# test_data.drop(test_data.index[drop_ind], inplace=True)
# test_data  


In [6]:
# train_data.to_csv('/content/drive/MyDrive/bigdata_deeplearning/data/re_train_data.csv', header = True, index = False)
# test_data.to_csv('/content/drive/MyDrive/bigdata_deeplearning/data/re_test_data.csv', header = True, index = False)

In [7]:
# 학습용 데이터셋 불러오기
import pandas as pd
train_data = pd.read_csv("/content/drive/MyDrive/bigdata_deeplearning/data/re_train_data.csv")
test_data = pd.read_csv("/content/drive/MyDrive/bigdata_deeplearning/data/re_test_data.csv")

In [8]:
train_data['카테고리'].value_counts()

3    1825
4    1777
2    1430
1    1326
0    1146
Name: 카테고리, dtype: int64

In [9]:
test_data['카테고리'].value_counts()

4    443
3    442
2    393
1    318
0    294
Name: 카테고리, dtype: int64

In [10]:
train_data["내용2"]

0       네 이번 뉴스를 보다 너무 어 이가 없어서 청원을 올립니 다 하루가 멀다 하고 올라...
1       안녕하세요 세 발달 지연 아이를 키우고 있는 대한민국의 평범한 어머니입니다 제가 청...
2       년전의 일이고 고소를 취하한 내용이지만 이자매가 그 일로 인해 자살하고 아버지마저 ...
3       은혜로 목사인 신 목사에게 꼬임을 당해 끌려간 사람들을 구하고 신 목사와 그 주변에...
4       최근 주 사이 우동기 대구시교육감에 대한 생각을 적지 않을 수 없어 몇 자 적습니다...
                              ...                        
7499    대입 수험생을 둔 가정에서 많은 분들이 아쉬워하는 교육과정평가원의 수능 및 모의평가...
7500    이번 국회의원 피기감 기관 지원과 관련 전수조사하여 잘못된 관행을 바로잡았으면 합니...
7501    울진 한울원전 호기는 상업운전 초기부터 심각한 전열관 손상이 있었습니다 그리고 년간...
7502    여러분 혹시 지금 서남대학교 폐교에 대한 기사를 보셨나요 죽전캠 분들에겐 해 당 사...
7503    이런 귀 중 한 문제는 만이 넘어가야 청원 승락되는 것에서 좀 컷트라인을 낮춰서 다...
Name: 내용2, Length: 7504, dtype: object

In [11]:
# 라벨링된 카테고리명 매핑 ex) {0: '경제민주화', 1: '교통/건축/국토', 2: '기타',3: '농산어촌', 4: '문화/예술/체육/언론'}
# mapping = dict(zip(range(len(encoder.classes_)), encoder.classes_))
# mapping

In [12]:
# Train / Test set 분리
# from sklearn.model_selection import train_test_split
# train, test = train_test_split(new_data, test_size=0.2, random_state=42)
# train, test = train_test_split(new_data, test_size=0.2, random_state=42)
# print("train shape is:", len(train))
# print("test shape is:", len(test))

In [13]:
# 기본 Bert tokenizer 사용
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

using cached model


In [14]:
class BERTDataset(Dataset):
    def __init__(self, dataset, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair) 
        self.sentences = [transform(i) for i in dataset['내용2']]
        self.labels = [np.int32(i) for i in dataset['카테고리']]
    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))
    def __len__(self):
        return (len(self.labels))
        

In [15]:
# Setting parameters
np.random.seed(1234)
max_len = 30 # 해당 길이를 초과하는 단어에 대해선 bert가 학습하지 않음
batch_size = 124
warmup_ratio = 0.1
num_epochs = 5
max_grad_norm = 1
log_interval = 200
learning_rate = 5e-5


In [16]:
bert_train = BERTDataset(train_data, tok, max_len, True, False)
bert_test = BERTDataset(test_data, tok, max_len, True, False)

In [17]:
print(bert_train)
print(bert_test)

In [18]:
# pytorch용 DataLoader 사용
train_dataloader = torch.utils.data.DataLoader(bert_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(bert_test, batch_size=batch_size, num_workers=5)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [19]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 18,
                 num_classes = 5, # softmax 사용 <- binary일 경우는 2
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()
    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [20]:
model = BERTClassifier(bertmodel, dr_rate=0.2).to(device)

In [21]:
# Prepare optimizer and schedule (linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

In [22]:
# 옵티마이저 선언
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss() # softmax용 Loss Function 정하기 <- binary classification도 해당 loss function 사용 가능
t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)
scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

In [23]:
# 학습 평가 지표인 accuracy 계산 -> 얼마나 타겟값을 많이 맞추었는가
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()
    return train_acc
# /max_indices.size()[0]

In [24]:
# # 모델 학습 시작
# for e in range(num_epochs):
#     train_acc = 0.0
#     # test_acc = 0.0
#     model.train()
#     for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm(train_dataloader)):
#         optimizer.zero_grad()
#         token_ids = token_ids.long().to(device)
#         segment_ids = segment_ids.long().to(device)
#         valid_length= valid_length
#         label = label.long().to(device)
#         out = model(token_ids, valid_length, segment_ids)
#         loss = loss_fn(out, label)
#         loss.backward
#         torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm) # gradient clipping
#         optimizer.step()
#         scheduler.step()  # Update learning rate schedule
#         train_acc += calc_accuracy(out, label)
#         if batch_id % log_interval == 0:
#             print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
#     print("epoch {} train acc {}".format(e+1, train_acc / (7847)))

In [25]:
# test

In [26]:
# unseen_test = pd.DataFrame([[test_sentence, test_label]], columns = [['제목', '카테고리']])
# unseen_values = unseen_test.values
# test_set = BERTDataset(unseen_values, 0, 1, tok, max_len, True, False)
# test_input = torch.utils.data.DataLoader(test_set, batch_size=1, num_workers=5)

In [27]:
test_input = torch.utils.data.DataLoader(test_data, batch_size=1, num_workers=5)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [28]:
# BERTDataset(unseen_values, tok, max_len, True, False)

In [29]:
# tqdm_notebook(test_input)

In [30]:
# # 모델 학습 시작
# test_acc = 0.0
# for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
#   optimizer.zero_grad()
#   token_ids = token_ids.long().to(device)
#   segment_ids = segment_ids.long().to(device)
#   valid_length= valid_length
#   label = label.long().to(device)
#   out = model(token_ids, valid_length, segment_ids)
#   loss = loss_fn(out, label)
#   loss.backward
#   torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm) # gradient clipping
#   optimizer.step()
#   scheduler.step()  # Update learning rate schedule
#   test_acc += calc_accuracy(out, label)
#   if batch_id % log_interval == 0:
#     print("batch id {} loss {} test acc {}".format(batch_id+1, loss.data.cpu().numpy(), test_acc / (batch_id+1)))

In [31]:
# 모델 학습 시작
for e in range(num_epochs):
    train_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm) # gradient clipping
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id == batch_size :
          print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id + 1)))    
    
# 

  0%|          | 0/61 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
  0%|          | 0/61 [00:00<?, ?it/s]

epoch 1 train acc 21.950819672131146


  0%|          | 0/61 [00:00<?, ?it/s]

epoch 2 train acc 21.459016393442624


  0%|          | 0/61 [00:00<?, ?it/s]

epoch 3 train acc 21.459016393442624


  0%|          | 0/61 [00:00<?, ?it/s]

epoch 4 train acc 21.57377049180328


100%|██████████| 61/61 [00:27<00:00,  2.23it/s]

epoch 5 train acc 21.770491803278688


In [32]:
model.eval() # 평가 모드로 변경
test_acc = 0.0

for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
  token_ids = token_ids.long().to(device)
  segment_ids = segment_ids.long().to(device)
  valid_length= valid_length
  label = label.long().to(device)
  out = model(token_ids, valid_length, segment_ids)
  test_acc += calc_accuracy(out, label)
  print("test acc {}".format(test_acc / (batch_id+1)))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  after removing the cwd from sys.path.


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


test acc 27.0
test acc 24.0
test acc 25.666666666666668
test acc 27.5
test acc 26.2
test acc 25.5
test acc 25.285714285714285
test acc 25.625
test acc 25.444444444444443
test acc 26.1
test acc 26.454545454545453
test acc 26.0
test acc 25.76923076923077
test acc 25.571428571428573
test acc 25.266666666666666
test acc 23.875

